<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">House Prices: Stochastic Gradient Descent (SGD)</h1>
</div>

Lesson: Stochastic Gradient Descent


|Notebook| MAE | LeaderBoard|
| --- | --- | --- |
|QuickStart|38341.2045|0.29234|
|Extra Features|32285.7959|0.24425|
|Features + Lasso|31349.8387|0.24425|
|Features + Ridge|31348.1429|0.24422|



In [1]:
# Black formatter https://black.readthedocs.io/en/stable/

! pip install nb-black > /dev/null

%load_ext lab_black

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Import Libraries</h1>
</div>

A best practise is to include all libraries here.  However, I will put a few imports farther down where they are first used so beginners can learn with an "as needed" approach.

In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from pathlib import Path

pd.options.display.max_columns = 100  # Want to view all the columns

In [3]:
# Black formatter https://black.readthedocs.io/en/stable/

! pip install nb-black > /dev/null

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Library</h1>
</div>

Creating a few functions that we will reuse in each project.

In [4]:
def read_data(path):
    data_dir = Path(path)

    train = pd.read_csv(data_dir / "train.csv")
    test = pd.read_csv(data_dir / "test.csv")
    submission_df = pd.read_csv(data_dir / "sample_submission.csv")

    print(f"train data: Rows={train.shape[0]}, Columns={train.shape[1]}")
    print(f"test data : Rows={test.shape[0]}, Columns={test.shape[1]}")
    return train, test, submission_df

In [5]:
def create_submission(model_name, target, preds):
    sample_submission[target] = preds
    if len(model_name) > 0:
        sample_submission.to_csv(f"submission_{model_name}.csv", index=False)
    else:
        sample_submission.to_csv(f"submission.csv", index=False)

    return sample_submission[:5]

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


def show_scores(gt, yhat):
    rmse = mean_squared_error(gt, yhat)
    mae = mean_absolute_error(gt, yhat)

    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Load Train/Test Data</h1>
</div>

- train.csv - Data used to build our machine learning model
- test.csv - Data used to build our machine learning model. Does not contain the target variable
- sample_submission.csv - A file in the proper format to submit test predictions

In [7]:
train, test, sample_submission = read_data(
    "../input/house-prices-advanced-regression-techniques"
)

train data: Rows=1460, Columns=81
test data : Rows=1459, Columns=80


In [8]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In supervised learning problems, we have a label or target.

In [9]:
TARGET = "SalePrice"

There are 79 features but to keep it simple we are only going to start with one.

In [10]:
FEATURES = ["GrLivArea", "LotArea", "TotalBsmtSF", "FullBath"]

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Missing Data</h1>
</div>

In [11]:
print(5 * "=", "Train", 5 * "=")
print(train[FEATURES].isnull().sum())
print(5 * "=", "Test", 5 * "=")
print(test[FEATURES].isnull().sum())

===== Train =====
GrLivArea      0
LotArea        0
TotalBsmtSF    0
FullBath       0
dtype: int64
===== Test =====
GrLivArea      0
LotArea        0
TotalBsmtSF    1
FullBath       0
dtype: int64


In [12]:
test["TotalBsmtSF"] = test["TotalBsmtSF"].fillna(0)

## Verify No Missing Data

In [13]:
print(test[FEATURES].isnull().sum())

GrLivArea      0
LotArea        0
TotalBsmtSF    0
FullBath       0
dtype: int64


In [14]:
y = train[TARGET]
X = train[FEATURES].copy()

X_test = test[FEATURES].copy()

In [15]:
X.head()

,GrLivArea,LotArea,TotalBsmtSF,FullBath
0,1710,8450,856,2
1,1262,9600,1262,2
2,1786,11250,920,2
3,1717,9550,756,1
4,2198,14260,1145,2


## Scale the Data

Doesn't make a difference so it's commented out.

In [16]:
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler = StandardScaler()

X = scaler.fit(X).transform(X)
X_test = scaler.transform(X_test)

In [17]:
X[:5]

array([[ 0.37033344, -0.20714171, -0.45930254,  0.78974052],
       [-0.48251191, -0.09188637,  0.46646492,  0.78974052],
       [ 0.51501256,  0.07347998, -0.31336875,  0.78974052],
       [ 0.38365915, -0.09689747, -0.68732408, -1.02604084],
       [ 1.2993257 ,  0.37514829,  0.19967971,  0.78974052]])

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Train Model with Train/Test Split</h1>
</div>

We split the training data so we can evaluate how well each model performs  We are saving 20% of the training data to validate the model(s).

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,  # Save 20% for validation
    random_state=42,  # Make the split deterministic
)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((1168, 4), (1168,), (292, 4), (292,))

# Create a Model

## SGDRegressor

- [Parameters](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

- penalty=('elasticnet', 'l1','l2')
- loss: ('squared_error’, ‘huber’, ‘epsilon_insensitive’,‘squared_epsilon_insensitive’)


In [19]:
%%time
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(
    n_iter_no_change=2000, loss='squared_error',
    penalty="l1", eta0=0.001, max_iter=200_000,
    learning_rate='adaptive',
    early_stopping=True,
)
model.fit(X_train, y_train)

# valid_preds = model.predict(poly_features.transform(X_valid))
valid_preds = model.predict(X_valid)
show_scores(y_valid, valid_preds)

MAE: 31333.9179
RMSE: 2407312425.6418
CPU times: user 5.92 s, sys: 5.15 ms, total: 5.92 s
Wall time: 5.97 s


In [20]:
# test_preds = model.predict(poly_features.transform(X_test))
test_preds = model.predict(X_test)
create_submission("sgd", TARGET, test_preds)

,Id,SalePrice
0,1461,115694.043894
1,1462,170319.630387
2,1463,192032.738577
3,1464,189182.317690
4,1465,189041.321725


## Fit/Train the model

In [21]:
%%time

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

model = SGDRegressor()

params = {'loss': ['squared_error','huber'],
          'max_iter': [500,1000,1500],
          'learning_rate': ['constant', 'optimal', 'invscaling'],
          'penalty': ['l2','l1','elasticnet']
          }

grid_search = GridSearchCV(model, params, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linea

CPU times: user 9.61 s, sys: 10.5 ms, total: 9.62 s
Wall time: 9.56 s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


GridSearchCV(cv=5, estimator=SGDRegressor(),
             param_grid={'learning_rate': ['constant', 'optimal', 'invscaling'],
                         'loss': ['squared_error', 'huber'],
                         'max_iter': [500, 1000, 1500],
                         'penalty': ['l2', 'l1', 'elasticnet']},
             scoring='neg_mean_absolute_error')

In [22]:
grid_search.best_params_

{'learning_rate': 'invscaling',
 'loss': 'squared_error',
 'max_iter': 1000,
 'penalty': 'l1'}

In [23]:
grid_search.best_estimator_

SGDRegressor(penalty='l1')

In [24]:
final_model = grid_search.best_estimator_
valid_preds = final_model.predict(X_valid)
show_scores(y_valid, valid_preds)

MAE: 31200.8301
RMSE: 2353584438.8593


In [25]:
test_preds = final_model.predict(X_test)

create_submission("sgd_cv", TARGET, test_preds)

,Id,SalePrice
0,1461,114961.020397
1,1462,171948.289065
2,1463,192652.786207
3,1464,189312.700395
4,1465,189060.631318
